<a href="https://colab.research.google.com/github/Vaycold/tensorflow_tutorial/blob/main/%239.CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# tf.data.Dataset로 csv 데이터 로드 

In [1]:
import functools

import numpy as np
import tensorflow as tf

In [2]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file('train_csv',TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file('eval_csv',TEST_DATA_URL)

16384/13049 [=====================================] - 0s 0us/step


In [3]:
np.set_printoptions(precision=3, suppress=True)

In [4]:
# Data load

! head {train_file_path}

survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.25,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
1,female,26.0,0,0,7.925,Third,unknown,Southampton,y
1,female,35.0,1,0,53.1,First,C,Southampton,n
0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y
0,male,2.0,3,1,21.075,Third,unknown,Southampton,n
1,female,27.0,0,2,11.1333,Third,unknown,Southampton,n
1,female,14.0,1,0,30.0708,Second,unknown,Cherbourg,n
1,female,4.0,1,1,16.7,Third,G,Southampton,n


In [5]:
label_column = 'survived'
labels = [0,1]

# 데이터셋 작성

def get_datasets(file_path, **kwargs) :
    dataset = tf.data.experimental.make_csv_dataset(
        file_path, 
        batch_size = 5,
        label_name = label_column,
        na_value = '?',
        num_epochs = 1,
        ignore_errors = True,
        **kwargs
    )
    return dataset

raw_train_data = get_datasets(train_file_path)
raw_test_data = get_datasets(test_file_path)

In [6]:
def show_batch(dataset) :
    for batch, label in dataset.take(1) :
        for key, value in batch.items() :
            print('{:20s} : {} '.format(key,value.numpy()))

In [7]:
show_batch(raw_train_data)

sex                  : [b'female' b'female' b'male' b'male' b'male'] 
age                  : [16. 25. 28. 22. 28.] 
n_siblings_spouses   : [0 1 0 0 0] 
parch                : [0 0 0 0 0] 
fare                 : [7.75  7.925 0.    7.796 7.896] 
class                : [b'Third' b'Third' b'Second' b'Third' b'Third'] 
deck                 : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown'] 
embark_town          : [b'Queenstown' b'Southampton' b'Southampton' b'Southampton' b'Southampton'] 
alone                : [b'y' b'n' b'y' b'y' b'y'] 


In [8]:
# 만약 첫 번째 줄에 열 이름이 포함되어 있지 않은 경우 column_names 인수로 전달 *이 경우에는 해당 x
csv_columns = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

temp_dataset = get_datasets(train_file_path, column_names = csv_columns)

show_batch(temp_dataset)

sex                  : [b'male' b'male' b'male' b'male' b'male'] 
age                  : [20. 36. 28. 28. 31.] 
n_siblings_spouses   : [0 1 1 0 0] 
parch                : [0 0 1 0 0] 
fare                 : [ 7.05  15.55  15.246  7.775 50.496] 
class                : [b'Third' b'Third' b'Third' b'Third' b'First'] 
deck                 : [b'unknown' b'unknown' b'unknown' b'unknown' b'A'] 
embark_town          : [b'Southampton' b'Southampton' b'Cherbourg' b'Southampton' b'Southampton'] 
alone                : [b'y' b'n' b'n' b'y' b'y'] 


In [9]:
select_columns = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']

temp_dataset = get_datasets(train_file_path, select_columns = select_columns)
show_batch(temp_dataset)

age                  : [15. 25. 28. 29. 22.] 
n_siblings_spouses   : [1 1 8 0 0] 
class                : [b'Third' b'First' b'Third' b'First' b'Third'] 
deck                 : [b'unknown' b'B' b'unknown' b'B' b'unknown'] 
alone                : [b'n' b'n' b'n' b'y' b'y'] 


In [10]:
# Data preprocessing

select_columns = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
default = [0, 0.0, 0.0, 0.0, 0.0]

temp_dataset = get_datasets(train_file_path,
                            select_columns = select_columns,
                            column_defaults = default)
show_batch( temp_dataset)

age                  : [28. 22. 51. 24. 19.] 
n_siblings_spouses   : [3. 0. 1. 0. 3.] 
parch                : [1. 0. 0. 0. 2.] 
fare                 : [ 25.467   7.896  77.958  69.3   263.   ] 


In [11]:
example_batch, labels_batch = next(iter(temp_dataset))

In [12]:
def pack(features, label) :
    return tf.stack(list(features.values()), axis=-1), label_column

packed_dataset =  temp_dataset.map(pack)

for features, labels in packed_dataset.take(1) :
    print(features.numpy())
    print()
    print(labels.numpy())

[[37.     2.     0.     7.925]
 [ 4.     1.     1.    16.7  ]
 [47.     1.     0.    14.5  ]
 [28.     0.     0.     7.229]
 [25.     1.     0.    26.   ]]

b'survived'


In [13]:
show_batch(raw_train_data)

sex                  : [b'female' b'female' b'male' b'female' b'male'] 
age                  : [27.   28.   28.    0.75 28.  ] 
n_siblings_spouses   : [1 0 1 2 0] 
parch                : [0 0 0 1 0] 
fare                 : [13.858  7.55  82.171 19.258 47.1  ] 
class                : [b'Second' b'Third' b'First' b'Third' b'First'] 
deck                 : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown'] 
embark_town          : [b'Cherbourg' b'Southampton' b'Cherbourg' b'Cherbourg' b'Southampton'] 
alone                : [b'n' b'y' b'n' b'n' b'y'] 


In [14]:
example_batch, labels_batch = next(iter(temp_dataset))

In [15]:
# 숫자 특성 목록을 선택하고 단일 열로 묶는 것보다 일반적인 전처리기를 정의함

class PackNumericFeatures(object) :
    def __init__(self, names) :
        self.names = names
    
    def __call__(self, features, labels) :
        numeric_features = [features.pop(name) for name in self.names]
        numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
        numeric_features = tf.stack(numeric_features, axis=-1)
        features['numeric'] = numeric_features

        return features, labels

In [16]:
NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

show_batch(packed_train_data)

sex                  : [b'male' b'male' b'female' b'female' b'male'] 
class                : [b'Second' b'Third' b'Second' b'Third' b'Third'] 
deck                 : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown'] 
embark_town          : [b'Southampton' b'Southampton' b'Southampton' b'Cherbourg' b'Southampton'] 
alone                : [b'n' b'y' b'y' b'n' b'y'] 
numeric              : [[60.     1.     1.    39.   ]
 [51.     0.     0.     7.054]
 [35.     0.     0.    21.   ]
 [ 0.75   2.     1.    19.258]
 [51.     0.     0.     8.05 ]] 


In [17]:
example_batch, labels_batch = next(iter(packed_train_data))

In [18]:
# 데이터 정규화

import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [19]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [20]:
def normalize(data, mean, std) :
    return (data-mean) / std

In [21]:
normalizer = functools.partial(normalize, mean = MEAN, std = STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn = normalizer,
                                                  shape = [len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_columns


[NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize at 0x7f97829f3950>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))]

In [22]:
example_batch['numeric']

<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[21.   ,  0.   ,  0.   ,  8.05 ],
       [28.   ,  1.   ,  0.   , 15.5  ],
       [35.   ,  0.   ,  0.   ,  7.05 ],
       [20.   ,  0.   ,  0.   ,  9.225],
       [16.   ,  0.   ,  0.   ,  7.733]], dtype=float32)>

In [23]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[-0.69 , -0.474, -0.479, -0.482],
       [-0.13 ,  0.395, -0.479, -0.346],
       [ 0.429, -0.474, -0.479, -0.501],
       [-0.77 , -0.474, -0.479, -0.461],
       [-1.089, -0.474, -0.479, -0.488]], dtype=float32)

In [24]:
# 범주형 데이터
# - tf.feature.column API 사용하여 각 범주열에 대해 tf.feature_column.indicator_column 을 가진 모음을 작성

CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}

In [25]:
categorical_columns = []

for feature, vocab in CATEGORIES.items() :
    cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key = feature, vocabulary_list = vocab
    )
    categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [26]:
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [27]:
categorical_layers = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layers(example_batch).numpy()[0])

[1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]


In [28]:
# 결합된 전처리 레이어

# 두개의 특성 열 모음을 추가하고 tf.keras.layers.DenseFeatures에 전달하여 두 입력 유형 추출, 전처리할 입력 레이어를 만들자

preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns + numeric_columns)

print(preprocessing_layer(example_batch).numpy()[0])

[ 1.     0.     0.     0.     1.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.    -0.69  -0.474
 -0.479 -0.482  1.     0.   ]


In [30]:
# 모델

model = tf.keras.Sequential([
            preprocessing_layer,
            tf.keras.layers.Dense(128, activation = 'relu'),
            tf.keras.layers.Dense(128, activation = 'relu'),
            tf.keras.layers.Dense(1)
])

model.compile(loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer = 'adam',
              metrics = ['accuracy'])

In [31]:
# Train

train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

model.fit(train_data, epochs = 20)

Epoch 1/20
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
126/126 [==============================] - 1s 2ms/step - loss: 0.4998 - accuracy: 0.7480
Epoch 2/20
126/126 [==============================] - 0s 2ms/step - loss: 0.4216 - accuracy: 0.8198
Epoch 3/20
126/126 [==============================] - 0s 2ms/step - loss: 0.4043 - accuracy: 0.8293
Epoch 4/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3911 - accuracy: 0.8309
Epoch 5/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3796 - accuracy: 0.8389
Epoch 6/20
126/126 [==============================] - 0s 1ms/step - loss: 0.3735 - accuracy: 0.8421
Epoch 7/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3700 - accuracy: 0.8357
Epoch 8/20
126/126 [==============================] - 0s 1ms/step - loss: 0.3552 - accuracy: 0.8485
Epoch 9/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3519 - accurac

In [32]:
test_loss, test_acc = model.evaluate(test_data)

Consider rewriting this model with the Functional API.
53/53 [==============================] - 0s 1ms/step - loss: 0.4564 - accuracy: 0.8447


In [34]:
print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_acc))



Test Loss 0.4563959836959839, Test Accuracy 0.8446969985961914


In [35]:
# label 유추

predictions = model.predict(test_data)

for predictions, survived in zip(predictions[:10], list(test_data)[0][1][:10]) :
    prediction = tf.sigmoid(predictions).numpy()
    print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))

Consider rewriting this model with the Functional API.
Predicted survival: 44.69%  | Actual outcome:  DIED
Predicted survival: 99.95%  | Actual outcome:  DIED
Predicted survival: 60.28%  | Actual outcome:  SURVIVED
Predicted survival: 92.56%  | Actual outcome:  DIED
Predicted survival: 69.28%  | Actual outcome:  DIED
